# Python Data Engineer Code Challenge

[Descripción]

In [6]:
import csv
import pandas as pd
import json
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine, Column, Integer, String, Boolean, Date
from sqlalchemy import func, text

In [6]:
csv_file = "data/candidates.csv"
df = pd.read_csv(csv_file, sep=";")
print(df[0:5])
print(len(df))

   First Name   Last Name                      Email Application Date  \
0  Bernadette   Langworth        leonard91@yahoo.com       2021-02-26   
1      Camryn    Reynolds        zelda56@hotmail.com       2021-09-09   
2       Larue      Spinka   okey_schultz41@gmail.com       2020-04-14   
3        Arch      Spinka     elvera_kulas@yahoo.com       2020-10-01   
4       Larue  Altenwerth  minnie.gislason@gmail.com       2020-05-20   

   Country  YOE  Seniority                         Technology  \
0   Norway    2     Intern                      Data Engineer   
1   Panama   10     Intern                      Data Engineer   
2  Belarus    4  Mid-Level                     Client Success   
3  Eritrea   25    Trainee                          QA Manual   
4  Myanmar   13  Mid-Level  Social Media Community Management   

   Code Challenge Score  Technical Interview Score  
0                     3                          3  
1                     2                         10  
2          

We can tell that there's not a column that tells us whether the candidate was hired or not, however, we have the right information to tell. In order to make future queries easier, we are going to create a new column named 'hired' later.

### Step II: Establishing a connection to the database

In [9]:
with open('./db_config.json', 'r') as file:
    data = json.load(file)

engine = create_engine(f'mysql+mysqlconnector://{data["user"]}:{data["password"]}@{data["host"]}:{data["server"]}/{data["db"]}')
base = declarative_base()


C:\Users\Mrìa Angelica\AppData\Local\Temp\ipykernel_8888\1044516210.py:5: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  base = declarative_base()


### Step III: Creating the table and modifying columns